In [14]:
# 런타임 초기화 후 제일 먼저
import gc
gc.collect()

20

In [5]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce

In [6]:
train_8 = pd.read_csv('train_8_성과정보_98이상제거_결측치처리.csv')
segment = pd.read_csv("Segment_병합.csv")

In [7]:
# 1️⃣ concat으로 segment 컬럼 붙이기
train_full = pd.concat([train_8, segment['Segment']], axis=1)

In [8]:
train_full

,기준년월,ID,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용금액_신용_전월,증감율_이용금액_신판_전월,...,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M,Segment
0,201807.0,TRAIN_000000,0.454548,0.454548,0.364727,1.999996,0.0,0.0,0.409766,0.409766,...,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543,D
1,201807.0,TRAIN_000001,-0.511808,-0.511808,-0.511808,0.000000,0.0,0.0,-1.010817,-1.010817,...,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000,E
2,201807.0,TRAIN_000002,-0.062716,-0.062716,0.067843,-1.999996,0.0,0.0,-0.902166,-0.902166,...,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420,C
3,201807.0,TRAIN_000003,0.135707,0.135707,0.153975,-1.999996,0.0,0.0,-0.866052,-0.866052,...,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124,D
4,201807.0,TRAIN_000004,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000028,0.000028,...,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,0.746919,0.923522,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E
2399996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D
2399997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C
2399998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E


In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [10]:
target = train_full['Segment']
X = train_full.drop(columns=['ID', 'Segment'])

In [11]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [12]:
from sklearn.preprocessing import LabelEncoder

# 문자열 target → 숫자형으로 변환
le = LabelEncoder()
target_encoded = le.fit_transform(target)

print("\n✅ 타겟 라벨 인코딩 완료")
print(pd.Series(target_encoded).value_counts())


✅ 타겟 라벨 인코딩 완료
4    1922052
3     349242
2     127590
0        972
1        144
Name: count, dtype: int64


In [13]:
corrs = X_encoded.corrwith(pd.Series(target_encoded))
corrs_sorted = corrs.abs().sort_values(ascending=False)

print("\n✅ 상관계수 (절대값 내림차순):")
print(corrs_sorted)


✅ 상관계수 (절대값 내림차순):
잔액_신판ca최대한도소진율_r6m    0.271469
잔액_신판ca최대한도소진율_r3m    0.263728
잔액_신판ca평균한도소진율_r3m    0.258409
잔액_신판ca평균한도소진율_r6m    0.254132
잔액_신판최대한도소진율_r6m      0.203843
잔액_신판최대한도소진율_r3m      0.196737
잔액_신판평균한도소진율_r3m      0.191232
잔액_신판평균한도소진율_r6m      0.184439
변동률_RV일시불평잔           0.104826
변동률_RVCA평잔            0.104093
증감율_이용건수_할부_전월        0.075308
혜택수혜율_B0M             0.072668
혜택수혜율_R3M             0.071983
변동률_카드론평잔             0.070965
증감율_이용금액_일시불_전월       0.063598
증감율_이용건수_CA_전월        0.060180
증감율_이용금액_신판_전월        0.060041
증감율_이용건수_체크_전월        0.057584
변동률_CA평잔              0.041461
증감율_이용금액_신용_전월        0.039801
증감율_이용금액_CA_분기        0.035653
증감율_이용금액_신용_분기        0.035309
증감율_이용건수_할부_분기        0.034114
증감율_이용금액_신판_분기        0.033650
변동률_일시불평잔             0.033614
증감율_이용금액_할부_분기        0.028543
증감율_이용금액_일시불_분기       0.026555
증감율_이용건수_CA_분기        0.024597
증감율_이용금액_체크_전월        0.022876
변동률_할부평잔              0.022191
증감율_이용건수_일시불_분기       0.020148
증감율_이용건수_카드론_분기    

In [15]:
# 1. 피처와 타깃 분리
target = train_full['Segment']
X = train_full.drop(columns=['ID', 'Segment'])

# 2. 인코딩 (One-Hot)
X_encoded = pd.get_dummies(X, drop_first=True)


In [16]:
print("X_encoded shape:", X_encoded.shape)
X_encoded.head()

X_encoded shape: (2400000, 46)


,기준년월,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용금액_신용_전월,증감율_이용금액_신판_전월,증감율_이용금액_일시불_전월,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,201807.0,0.454548,0.454548,0.364727,1.999996,0.0,0.0,0.409766,0.409766,0.313300,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543
1,201807.0,-0.511808,-0.511808,-0.511808,0.000000,0.0,0.0,-1.010817,-1.010817,-1.010817,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000
2,201807.0,-0.062716,-0.062716,0.067843,-1.999996,0.0,0.0,-0.902166,-0.902166,0.289565,...,1.006124,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420
3,201807.0,0.135707,0.135707,0.153975,-1.999996,0.0,0.0,-0.866052,-0.866052,0.276947,...,0.999998,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124
4,201807.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000028,0.000028,0.000028,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,0.746919,0.923522


In [18]:
import numpy as np

# NaN 존재 여부 확인
print("✅ NaN 있는 컬럼 개수:", X_encoded.isnull().sum().gt(0).sum())

# inf 존재 여부 확인
print("⚠️ inf 있는 값 개수:", np.isinf(X_encoded).sum().sum())


✅ NaN 있는 컬럼 개수: 46
⚠️ inf 있는 값 개수: 0


In [19]:
null_cols = X_encoded.isnull().sum()
null_cols = null_cols[null_cols > 0].sort_values(ascending=False)
print("❗ 결측치 있는 컬럼들:")
print(null_cols)

❗ 결측치 있는 컬럼들:
기준년월                  1351425
잔액_신판ca최대한도소진율_r3m    1351425
증감율_이용금액_체크_분기        1351425
증감율_이용금액_카드론_분기       1351425
잔액_신판평균한도소진율_r6m      1351425
잔액_신판최대한도소진율_r6m      1351425
잔액_신판평균한도소진율_r3m      1351425
잔액_신판최대한도소진율_r3m      1351425
잔액_신판ca평균한도소진율_r6m    1351425
잔액_신판ca최대한도소진율_r6m    1351425
잔액_신판ca평균한도소진율_r3m    1351425
변동률_일시불평잔             1351425
증감율_이용건수_신용_전월        1351425
변동률_RV일시불평잔           1351425
변동률_할부평잔              1351425
변동률_CA평잔              1351425
변동률_RVCA평잔            1351425
변동률_카드론평잔             1351425
변동률_잔액_B1M            1351425
변동률_잔액_일시불_B1M        1351425
변동률_잔액_CA_B1M         1351425
혜택수혜율_R3M             1351425
증감율_이용금액_CA_분기        1351425
증감율_이용금액_할부_분기        1351425
증감율_이용금액_일시불_분기       1351425
증감율_이용금액_신판_분기        1351425
증감율_이용건수_신판_전월        1351425
증감율_이용건수_일시불_전월       1351425
증감율_이용건수_할부_전월        1351425
증감율_이용건수_CA_전월        1351425
증감율_이용건수_체크_전월        1351425
증감율_이용금액_신용_전월        1351425
증감율_이용금액_신판_전월        1351